# In this notebook I will solve the loan prediction problem. The idea is to automate who gets the load and who does not.

In [1]:
import numpy as np
import pandas as pd
import decimal

In [2]:
trainSet = pd.read_csv('./train_Loan.csv') #loading the training set
testSet = pd.read_csv('./test_Loan.csv') # loading the test set

In [3]:
trainSet.head(5) #take a quick look at the training set

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0,NaN,360,1,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0,141,360,1,Urban,Y


In [4]:
testSet.head(5) #take a look at the test set

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110,360,1,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126,360,1,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208,360,1,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100,360,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78,360,1,Urban


In [5]:
trainSet.shape #to see how many rows and columns it have

(614, 13)

In [6]:
testSet.shape

(367, 12)

In [7]:
trainSet.isnull().sum() #to see how many null values are in each column

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
trainSet.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [9]:
trainSet[pd.isnull(trainSet).any(axis=1)] #it prints out the rows with null values


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0,NaN,360,1,Urban,Y
11,LP001027,Male,Yes,2,Graduate,NaN,2500,1840,109,360,1,Urban,Y
16,LP001034,Male,No,1,Not Graduate,No,3596,0,100,240,NaN,Urban,Y
19,LP001041,Male,Yes,0,Graduate,NaN,2600,3500,115,NaN,1,Urban,Y
23,LP001050,NaN,Yes,2,Not Graduate,No,3365,1917,112,360,0,Rural,N
24,LP001052,Male,Yes,1,Graduate,NaN,3717,2925,151,360,NaN,Semiurban,N
29,LP001087,Female,No,2,Graduate,NaN,3750,2083,120,360,1,Semiurban,Y
30,LP001091,Male,Yes,1,Graduate,NaN,4166,3369,201,360,NaN,Urban,N
35,LP001106,Male,Yes,0,Graduate,No,2275,2067,NaN,360,1,Urban,Y
36,LP001109,Male,Yes,0,Graduate,No,1828,1330,100,NaN,0,Urban,N


In [10]:
trainSet.isnull().values.ravel().sum() #total number of rows with null values

149

In [11]:
trainSet.shape[0]  # total number of rows
trainSet.isnull().values.ravel().sum() / float(trainSet.shape[0]) * 100 # how much data do we lose if I delete
    #rows with NaN values?

24.267100977198698

I will loose 24% of data! This is significant! I can do something else too. I can only remove the null values from rows that have binary values such as credit history. For continuous values I can replace the null values with the average values obtained. 

Another thing that I could do is to use algorithms to predict the NaN values. This of course takes much longer and I don't have time for that at this point. Maybe later! I would do it for more important projects and I think it is a good idea.

In [23]:
trainSet.dropna(subset = ['Gender','Married','Education','Self_Employed','Credit_History'] , inplace = True)

In [26]:
trainSet.isnull().values.ravel().sum()

45

In [27]:
trainSet['Loan_Amount_Term'].unique()

array([ 360.,  120.,   nan,  180.,   60.,  300.,  480.,  240.,   36.,   84.])

Now for Dependants, ApplicantIncome ,CoapplicantIncome , LoanAmount and Loan_Amount_Term I can replace them with mean values.